In [1]:
import pandas as pd
import numpy as np
import os, glob,re

from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt


import torch
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'

In [ ]:
!pip install transformers
!pip install pytorch_lightning

     |████████████████████████████████| 2.2MB 12.6MB/s 
     |████████████████████████████████| 870kB 39.8MB/s 
     |████████████████████████████████| 3.3MB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b579d67a8cf6ba6f6ed6efaf2e4c6049c31525766dc6294c3b061ec8735c483c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import transformers
import pytorch_lightning as pl

pl.seed_everything(13)

In [21]:
df = pd.read_csv(Path(PATH) / 'train.csv')
df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


Look for text


In [27]:
df1 = pd.read_json(Path(PATH) /'train' /'d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json')
df1

,section_title,text
0,What is this study about?,This study used data from the National Educati...
1,Features of Dual Enrollment Programs,Dual enrollment programs allow high school stu...
2,WWC Single Study Review,"What did the study find?\nThe study reported, ..."
3,WWC Rating,The research described in this report meets WW...
4,Intervention group,The intervention group was comprised of those ...
5,Comparison group,The comparison group was comprised of NELS par...
6,Outcomes and measurement,There were two outcomes examined in the study:...
7,Support for implementation,The study was based on secondary data from a n...
8,Reason for review,This study was identified for review by receiv...
9,Degree attainment,Any college degree attainment The study author...


for BERT not useble very length text, base model 512 tokens

In [32]:
df1.text.apply(lambda x: len(x.split())).describe().to_dict()

{'25%': 27.75,
 '50%': 41.5,
 '75%': 165.0,
 'count': 18.0,
 'max': 298.0,
 'mean': 98.0,
 'min': 10.0,
 'std': 95.73615703957353}

In [38]:
len_text = len(df1.text.str.cat(sep = '\n').split())
len_text

1764

In [44]:
def see_length(file: Path):
    # make conf interval for length
    tmp_df = pd.read_json(file)
    tmp_stat = tmp_df.text.apply(lambda x: len(x.split())).describe().to_dict()
    len_text = len(tmp_df.text.str.cat(sep = '\n').split())
    id = file.stem
    return {
        'id':id,
        'min': tmp_stat['min'],
        'mean': tmp_stat['mean'],
        'max': tmp_stat['max'],
        'len_txt': len_text
        }

In [45]:
a = Path(PATH) /'train' /'d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json'
see_length(a)

{'id': 'd0fa7568-7d8e-4db9-870f-f9c6f668c17b',
 'len_txt': 1764,
 'max': 298.0,
 'mean': 98.0,
 'min': 10.0}

In [48]:
list_train_json = list(Path(PATH).glob('train/*.json'))

In [52]:
tmp = []
for file in tqdm(list_train_json):
    tmp_dict = see_length(file)
    tmp.append(
        tmp_dict
        )

In [59]:
df = pd.DataFrame(tmp)
df.head()

,id,min,mean,max,len_txt
0,f0249680-2273-4b1a-8a0a-2e8bbf9ec11a,0.0,316.541667,1197.0,15194
1,ef4b7964-b7dc-4570-804e-74cc7b804c61,42.0,427.266667,985.0,6409
2,ec938f38-37e7-4a35-87bd-e59ffa02d52e,76.0,318.214286,532.0,4455
3,f042f6d8-c940-4cda-9b96-75cb95db4ef8,0.0,378.200000,705.0,3782
4,ed1a995e-63f9-4542-b515-bcaab0ed4295,0.0,380.789474,2738.0,7235


In [65]:
df.mean(axis = 0)

min         553.132858
mean        924.293620
max        1864.406259
len_txt    7049.041771
dtype: float64

In [103]:
df[df['min'] < 100].mean(axis = 0)

min          10.655568
mean        374.762448
max        1352.718951
len_txt    6855.988687
dtype: float64

In [115]:
df[df['min'] == 6]

,id,min,mean,max,len_txt
35,e9057a7a-ea41-4b40-88bf-3bfb8354c807,6.0,591.400000,1359.0,5914
146,f7e96f4d-3556-4f07-990a-dc4a069b4c93,6.0,316.750000,1518.0,6335
640,fd2e2c07-ac85-409f-88d4-846931276cba,6.0,458.875000,1210.0,7342
1462,dc0fc97d-19e7-4154-82e8-836b4cdd2d7d,6.0,363.400000,705.0,1817
1755,ed2ce29d-5a01-431c-80c9-40fa104fb04f,6.0,305.333333,1033.0,1832
1963,e70843e1-b397-4936-9d43-9823dc599edb,6.0,547.600000,990.0,2738
2022,ce6402e8-5182-42ef-96ac-3b9f703c2dee,6.0,431.666667,1088.0,2590
2293,d2a27799-80ab-43f9-9333-0806821ec14f,6.0,193.000000,576.0,2895
2330,cc81652d-51b2-49b2-be7a-1842125a698b,6.0,1821.800000,4253.0,9109
2443,cbb78840-b76f-4bbb-b37c-4636d1a7fd6b,6.0,503.916667,1253.0,6047


In [ ]:
найти количество пропусков в тексте в титле и количество титлов
количество текста в одну Future Research .................................
количество CHAPTER  Page
минимальная длина текста 10
минимальная длина титлов>0 нет точек и других знаков(исключение -) только 
"www.balticsportscience.com 


In [70]:
f = 'f0249680-2273-4b1a-8a0a-2e8bbf9ec11a.json'
df2 = pd.read_json(Path(PATH) /'train' / f)
df2

,section_title,text
0,,Future Research .................................
1,LIST OF FIGURES,
2,Page,Significant moderation effects were found for ...
3,CHAPTER 1. INTRODUCTION,American Indian and Alaska Native (AIAN) child...
4,CHAPTER 2. LITERATURE REVIEW,Children that have not mastered critical acade...
5,Pre-Academic Skills,Researchers consistently point to the importan...
6,Early Reading Skills,Language development is the process by which c...
7,Early Math Skills,Cognitive skills are the result of exposure to...
8,Behavioral Skills,Early behavioral skills provide a critical fou...
9,Social Competence,Social competence may be the most broadly defi...


In [78]:
df2.text.replace('', np.nan, inplace=True)

In [81]:
df2.dropna()

,section_title,text
0,,Future Research .................................
2,Page,Significant moderation effects were found for ...
3,CHAPTER 1. INTRODUCTION,American Indian and Alaska Native (AIAN) child...
4,CHAPTER 2. LITERATURE REVIEW,Children that have not mastered critical acade...
5,Pre-Academic Skills,Researchers consistently point to the importan...
6,Early Reading Skills,Language development is the process by which c...
7,Early Math Skills,Cognitive skills are the result of exposure to...
8,Behavioral Skills,Early behavioral skills provide a critical fou...
9,Social Competence,Social competence may be the most broadly defi...
10,Approaches to Learning,Another foundational skill connecting children...


In [82]:
'Future Research .................................'.split()

['Future', 'Research', '.................................']

In [99]:
df['min'].value_counts().sort_values(ascending=False).head(30)

0.0     9052
15.0      79
26.0      77
2.0       74
1.0       70
11.0      68
7.0       68
10.0      67
9.0       65
13.0      62
12.0      61
16.0      57
14.0      54
3.0       54
17.0      50
8.0       50
6.0       48
21.0      48
5.0       47
19.0      47
42.0      47
40.0      46
18.0      46
20.0      44
35.0      43
32.0      42
36.0      42
28.0      41
4.0       41
22.0      41
Name: min, dtype: int64

In [87]:
df.iloc[7215]

id         755506a3-1516-4c8b-8fc0-ca10027aed61
min                                           1
mean                                        379
max                                        1231
len_txt                                   11749
Name: 7215, dtype: object

In [88]:
f = '755506a3-1516-4c8b-8fc0-ca10027aed61.json'
df3 = pd.read_json(Path(PATH) /'train' / f)
df3

,section_title,text
0,,There are a number of threats to coastal areas...
1,EPA Coastal Wetland Regional Reviews,EPA conducted these CWRs to identify and bette...
2,"Conduct a review of current, readily available...","For the selected review area, the Coastal Wetl..."
3,Conduct stakeholder discussions.,"EPA sought an entity to serve as the ""host"" of..."
4,Assemble a coastal wetland regional review sum...,Once the notes from the stakeholder discussion...
5,4,Questions posed during stakeholder discussions...
6,2.,3.
7,4.,What are the root causes of coastal wetland lo...
8,Mid-Atlantic Review,The Mid-Atlantic region hosts a wide variety o...
9,Mid-Atlantic Coastal Wetland Stressors,"Historically, coastal wetlands in the Mid-Atla..."


In [105]:
f = '9e7a78df-4757-40d8-b347-fe6578f0e4ad.json'
df4 = pd.read_json(Path(PATH) /'train' / f)
df4

,section_title,text
0,,a1111111111 a1111111111 a1111111111 a111111111...
1,Introduction,"Coronavirus disease , caused by severe acute r..."
2,Ethics statement,This study was approved by the University Mala...
3,Clinical sample collection,Patients admitted to UMMC with suspected COVID...
4,Whole genome sequencing analysis,Viral RNA was extracted from 58 positive clini...
5,Bioinformatic analysis,The sequenced reads were analysed using Geneio...
6,Results,Fifty-eight whole genome sequences with >99% r...
7,Discussion,"In Malaysia, the Tablighi Jamaat mass gatherin..."


In [107]:
df4.section_title.values

array(['', 'Introduction', 'Ethics statement',
       'Clinical sample collection', 'Whole genome sequencing analysis',
       'Bioinformatic analysis', 'Results', 'Discussion'], dtype=object)

In [122]:
f = '6bfda128-a0c0-41ba-96a8-ec62731124d9.json'
df5 = pd.read_json(Path(PATH) /'train' / f)
df5#.iloc[3].values

,section_title,text
0,Careers of Korean Ph.Ds with Degrees of Foreign,Countries and the HRD Policy of the Highly Ski...
1,Overview,Due to the development of information and comm...
2,Main Results of the Research,A. The overall number of foreign doctoral hold...
3,C. Career activities after obtaining a doctora...,"At the time of receiving a doctorate degree, h..."
4,Policy Implications for Utilizing High Quality...,Who Earned a Doctorate Degree Abroad
5,A. Basic directions,The policies should be made with a basic under...
6,B. Policy recommendations for the utilization ...,1) Strategies to attract more doctoral recipie...


In [140]:
f = 'c6e74b14-6da5-43bf-9723-dcce7c2124ff.json'
df6 = pd.read_json(Path(PATH) /'train' / f)
df6

,section_title,text
0,,"Female doctorate recipients by broad field, 19..."
1,Figure 12,Doctorates earned by minority U.S. citizens by...
2,Figure 13,Median number of years to doctorate from bacca...
3,Figure 14,Age distribution at doctorate by broad field o...
4,Figure 15,Primary sources of financial support for docto...
5,LIST OF TABLES,Page Table 1 Doctorates awarded by U.S. colleg...
6,Trends in Doctorate Recipients,Overall Numbers and Rates of. Growth
7,"77.77-FT""","During the 1998 academic year (July 1, 1997, t..."
8,Doctorates by Broad Field,The SED classifies research doctoral degrees i...
9,Doctorates by Sex,The aggregate percentage increase in doctorate...


In [146]:
df6.iloc[7].values

array(['77.77-FT"',
       'During the 1998 academic year (July 1, 1997, through June 30, 1998), U.S. universities awarded a total of 42,683 research doctorate degrees,3 marking the thirteenth straight year in which the absolute number of doctorates increased. The annual growth rate from 1997 to 1998, 0.3 percent, was the same rate as for the preceding year.4 Over the past few years, the rate of increase has become markedly smaller than earlier in the decade. (See table 1.) In absolute numbers, 42,683 represents an increase of 2,882 doctorates over the number for 1993 and is 9,183 more than in 1988.5 For the 10-year interval between 1988 and 1998, U.S. universities collectively awarded almost 400,000 doctorates (397,048), as compared to a total of 316,413 for the preceding 10-year period. U.S. institutions have awarded more than one million doctorates (1,174,442) over the last 40 years, of which 33.8 percent were granted within the last 10 years (figures 1 and 2). The aggregate figure 

In [141]:
f = 'e9057a7a-ea41-4b40-88bf-3bfb8354c807.json'
df7 = pd.read_json(Path(PATH) /'train' / f)
df7

,section_title,text
0,I. Introduction,"In the United States, there has been an increa..."
1,4,Our empirical strategy is discussed in section...
2,II. Policy Context,The federal food and nutritional safety net is...
3,III. Data,This study uses data from the Early Childhood ...
4,IV. Empirical Strategy,This paper uses longitudinal data to explore d...
5,V. Potential Manipulation,The validity of our regression discontinuity d...
6,VI. Results,We begin by presenting descriptive statistics ...
7,VII. Extensions and robustness checks,In this section we present the stability of ou...
8,VIII. Discussion,"In this paper, we explore the effect of aging ..."
9,Figure 1 -Graphical Inspection,"* p<.1, ** p<.05, *** p<.01."


In [154]:
df7.iloc[2].values

array(['II. Policy Context',
       "The federal food and nutritional safety net is currently a patchwork. Program services may be delivered in the form of vouchers, (near) cash supplements, or directly as food. Services may be available to specific members of the household only or to the entire household. In addition to household income eligibility, children's eligibility for a specific program may depend upon their age and the income level of others in their day-care or school. The result of this hodge-podge of food and nutritional programs is that different households with similar income levels and numbers of children, may be receiving substantially different bundles of food assistance. While variation may occur across the entire childhood period, there is a significant transition in the types of food and nutrition programs for which children qualify as children reach age five and become eligible to enter kindergarten. The main federal food and nutrition assistance program that targ

In [142]:
df_head[df_head['Id'] == 'e9057a7a-ea41-4b40-88bf-3bfb8354c807']

,Id,pub_title,dataset_title,dataset_label,cleaned_label
13879,e9057a7a-ea41-4b40-88bf-3bfb8354c807,Families with Hungry Children and the Transiti...,Early Childhood Longitudinal Study,Early Childhood Longitudinal Study,early childhood longitudinal study


['77.77-FT"']

In [152]:
def clean_text(txt: str)-> str:
    return re.sub('[^A-Za-z]+', ' ', str(txt).lower()).strip()

In [153]:
clean_text('77.77-FT"')

'ft'

In [155]:
clean_text('II. Policy Context',)

'ii policy context'

In [156]:
clean_text('a 0 g`v)')

'a g v'

In [158]:
df_head.iloc[19658].cleaned_label

'cas covid 19 antiviral candidate compounds dataset'

У НАС ЕСТЬ ПРОПУСКИ В в тексте и в названии
есть скобки
надо оставить только текст цифры мне тоже не нужны

карочь просто пипец в данных

In [162]:
def have_digit(txt: str) -> bool:
    return any(char.isdigit() for char in txt)

reusult = df_head.cleaned_label.apply(lambda x: have_digit(x))

In [168]:
for i in df_head[reusult].cleaned_label.values:
    if 'cov' not in i:
        print(i)

In [ ]:
txt = df1.text.str.cat(sep = '\n').lower()
txt[:100]

'this study used data from the national education longitudinal study (nels:88) to examine the effects'

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
txt = 'this study used data from the national education longitudinal study (nels:88) to examine the effects'
out_dict = tokenizer.encode_plus(
    txt,
    add_special_tokens = True,
    max_length = 24,
    padding = 'max_length',
    return_tensors = 'pt',
    truncation = True,
    return_attention_mask = True
)

In [ ]:
out_dict

{'input_ids': tensor([[  101,  2023,  2817,  2109,  2951,  2013,  1996,  2120,  2495, 20134,
          2817,  1006, 11265,  4877,  1024,  6070,  1007,  2000, 11628,  1996,
          3896,   102,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}

In [ ]:
voc = tokenizer.get_vocab()
voc = {v: k for (k, v) in voc.items()}

In [ ]:
[voc[i] for i in out_dict['input_ids'].tolist()[0]]

['[CLS]',
 'this',
 'study',
 'used',
 'data',
 'from',
 'the',
 'national',
 'education',
 'longitudinal',
 'study',
 '(',
 'ne',
 '##ls',
 ':',
 '88',
 ')',
 'to',
 'examine',
 'the',
 'effects',
 '[SEP]',
 '[PAD]',
 '[PAD]']

In [ ]:
len(df.Id.unique())

14316

не могу сообразить что указывать в таргет

в каждом json есть заголовки может исрользовать только их


передаем json и читаем по заголовкам
к примеру у нас в d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json 18 заголовков в каждом текст
есть ли смысл читать и по каждому что то вернуть а потом оптимизировать(тк усреднить)

искать кааой-то контекст по заголовку но их много

может сделать ответ на вопрос или поиск сущности

[Summary of the tasks](https://huggingface.co/transformers/task_summary.html#)

try:
- Masked Language Modeling
- Extractive Question Answering
- Summarization

Interest links:

https://huggingface.co/transformers/preprocessing.html
https://huggingface.co/transformers/tokenizer_summary.html

https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1
https://www.kaggle.com/tungmphung/pytorch-bert-for-named-entity-recognition




In [ ]:
"""
AutoModel.from_pretrained ('bert-base-cased')
создаст модель, которая является экземпляром BertModel.

AutoConfig
classtransformers.AutoConfig [ИСТОЧНИК]
Это общий класс конфигурации, который будет создан как один из классов конфигурации библиотеки при 
создании с помощью метода класса from_pretrained ().
"""

conf = transformers.AutoConfig.from_pretrained('bert-base-cased', num_labels =130)
model = transformers.AutoModel.from_pretrained('bert-base-cased', config = conf)

In [ ]:
model